In [ ]:
import ee
import geemap

In [ ]:
Map = geemap.Map()
Map

In [ ]:
point = ee.Geometry.Point([-122.4439, 37.7538])
# 建立一个点坐标
# point = ee.Geometry.Point([-87.7719, 41.8799])

# 导入landsat8数据
image = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR') \
    .filterBounds(point) \
    .filterDate('2019-01-01', '2019-12-31') \
    .sort('CLOUD_COVER') \
    .first() \
    .select('B[1-7]')

vis_params = {
    'min': 0,
    'max': 3000,
    'bands': ['B5', 'B4', 'B3']
}

Map.centerObject(point, 8)

In [ ]:
nlcd = ee.Image('USGS/NLCD/NLCD2016').select('landcover').clip(image.geometry())

In [ ]:
# 选取样点
points = nlcd.sample(**{
    'region': image.geometry(),
    'scale': 30,
    'numPixels': 5000,
    'seed': 0,
    'geometries': True
})

In [ ]:
Map.addLayer(image, {'bands':['B4', 'B3', 'B2'], 'max':3000, 'min':0}, 'rgb')
Map.addLayer(nlcd, {}, 'nlcd')
Map.addLayer(points, {}, 'training')

In [ ]:
label = 'landcover'

training = image.sampleRegions(**{
  'collection': points,
  'properties': [label],
  'scale': 30
})

trained = ee.Classifier.smileRandomForest(numberOfTrees=100)\
            .train(training, label)

In [ ]:
result = image.classify(trained)

In [ ]:
# 获取值
class_values = nlcd.get('landcover_class_values').getInfo()
# class_values
# 获取值对应的颜色
class_palette = nlcd.get('landcover_class_palette').getInfo()
# class_palette
# 改变图像对应的属性
landcover = result.set('classification_class_values', class_values)
landcover = landcover.set('classification_class_palette', class_palette)
Map.addLayer(landcover, {}, 'Land cover')
Map

In [ ]:
from Monarch.user_gee import clip_dow_merge
import os
out_name = 'E:/Paper/Data/result/test'
while not os.path.exists(out_name+'.tif'):
    clip_dow_merge(image.geometry(), result, out_name, 30)